In [1]:
import os
import json
import time
import glob
import requests
import numpy as np
import pandas as pd
from geopy.distance import vincenty

In [2]:
API_KEY = ''

keypoints = {'Broadway-Lafayette': (40.725297, -73.995109),
             'Union Square' : (40.735289, -73.990705),
             '59th and Lex' : (40.762855, -73.967956),
             'Atlantic Ave' : (40.683475, -73.975669),
             'Grand Central': (40.752698, -73.977506)}

station_map = {'318-320 Lafayette St, New York, NY 10012, USA':'Broadway-Lafayette',
               '131 E 60th St, New York, NY 10022, USA':'59th and Lex',
               '190 Park Ave Viaduct, New York, NY 10017, USA':'Grand Central',
               '623-625 Atlantic Ave, Brooklyn, NY 11217, USA':'Atlantic Ave',
               '4 Union Square E, New York, NY 10003, USA':'Union Square'}

destinations = map(lambda v: '{0},{1}'.format(*v), keypoints.values())
destinations = '|'.join(destinations)

template =  'https://maps.googleapis.com/maps/api/distancematrix/json?' \
            'units=metric&' \
            'origins={lat},{long}&' \
            'destinations={destinations}&' \
            'key={API_KEY}'

In [8]:
output_dir = '../data/keypoints/'
train=pd.read_json('../data/train.json')
train.source = 'train'
test=pd.read_json('../data/test.json')
test.source = 'test'
data = pd.concat([train, test]) 

existing_IDs = glob.glob(output_dir + '*.json')
existing_IDs = [os.path.basename(path).replace('.json', '') for path in existing_IDs]
existing_IDs = [int(listing_id) for listing_id in existing_IDs]
data = data[~data.listing_id.isin(existing_IDs)].iloc[0:5]

In [9]:
n = 0
for index, row in data.iterrows():
    apt_coord = (row.latitude, row.longitude)
    
    args = {'lat': apt_coord[0],
            'long': apt_coord[1],
            'destinations':destinations,
            'API_KEY':API_KEY}    
    
    url = template.format(**args)
    response = requests.get(url)
    response = json.loads(response.content)
    if response['status'] == 'REQUEST_DENIED': 
        print url, '\n', response
        break
        
    durations = response['rows'][0]['elements']
    stations  = map(lambda address: station_map[address], response['destination_addresses'])
    durations = map(lambda elem: {'station':elem[0],'dist':elem[1]['distance']['value'], 
                                  'time':elem[1]['duration']['value']},
                    zip(stations, durations))
    
    f = os.path.join(output_dir, '{0}.json'.format(row.listing_id))
    with open(f, 'w+') as f: json.dump(durations, f)

    n+=1
    if n == 3500:
        break